# Amazon Bedrock AgentCore Policy - NL2Cedar デモ

## 概要

Amazon Bedrock AgentCore Policy のハンズオンデモへようこそ！このノートブックでは、自然言語から Cedar ポリシーを生成する完全なワークフローをガイドします。このノートブックでは、生成可能な様々なタイプのポリシーを示し、様々なポリシー構造の理解方法を説明します。

### Amazon Bedrock AgentCore Policy の自然言語オーサリングとは？

NL2Cedar - Cedar ポリシーの自然言語オーサリングは、自然言語で認可要件を記述し、自動的に Cedar 構文に変換するとともに、生成されたポリシーが要件に一致することを検証します。

---

## 前提条件

開始する前に、以下を確認してください：

- AWS CLI が適切な認証情報で設定されていること
- Python 3.10+ と boto3 がインストールされていること
- `bedrock_agentcore_starter_toolkit` パッケージがインストールされていること
- AWS Lambda へのアクセス権限があること（ターゲット関数用）

01-Getting-Started/AgentCore-Policy-Demo.ipynb では、3 つの Lambda ターゲットを持つ保険引受ユースケース用の Gateway をセットアップしています。同じ Gateway セットアップを使用します。

それでは始めましょう！

---

# ステップ 0: 環境セットアップ

まず、環境を確認し、必要なライブラリをインポートします。

In [ ]:
%pip install -r requirements.txt

In [ ]:
# Import required libraries
import sys
import os
from pathlib import Path
import subprocess
import boto3
import json
import logging
import time

# Add the scripts directory to Python path
scripts_dir = Path.cwd() / "scripts"
if str(scripts_dir) not in sys.path:
    sys.path.insert(0, str(scripts_dir))

# Verify region
session = boto3.Session()
region = session.region_name or "us-east-1"

# Verify AWS credentials
try:
    sts = session.client("sts")
    identity = sts.get_caller_identity()
    print("✅ AWS 認証情報を確認しました")
    print(f"   アカウント: {identity['Account']}")
    print(f"   ユーザー/ロール: {identity['Arn']}")
except Exception as e:
    print(f"❌ AWS 認証情報エラー: {e}")
    print("   AWS CLI を設定してください: aws configure")

### 08-AgentCore-policy/01-Getting-Started/AgentCore-Policy-Demo.ipynb からの Gateway セットアップが存在するか確認。存在しない場合、次のステップで 3 つの Lambda ターゲットを持つ保険引受用の Gateway をセットアップします

In [ ]:
# Check if Gateway for Insurance Underwriting exists

# Get the path to the Getting-Started directory
current_dir = Path.cwd()
getting_started_dir = current_dir.parent / "01-Getting-Started"
config_file = getting_started_dir / "config.json"
scripts_dir = getting_started_dir / "scripts"

print("🔍 Checking for configuration...")
print(f"検索中: {config_file.relative_to(current_dir.parent.parent)}")

if config_file.exists():
    print("✅ Configuration file found!")

    # Optionally, verify it has the required fields
    import json

    try:
        with open(config_file, "r") as f:
            config = json.load(f)

        required_fields = ["lambdas", "gateway", "region"]
        missing_fields = [field for field in required_fields if field not in config]

        if missing_fields:
            print(f"⚠️  Warning: Config file is missing fields: {missing_fields}")
            print("   You may need to re-run the setup scripts.")
        else:
            print("✅ Configuration is complete!")

    except json.JSONDecodeError:
        print("⚠️  Warning: Config file exists but is not valid JSON")

else:
    print("❌ Configuration file not found!")
    print("\n" + "=" * 70)
    print("Setting up infrastructure...")
    print("=" * 70)

    # Change to the Getting-Started directory to run scripts
    os.chdir(getting_started_dir)

    try:
        # Step 1: Deploy Lambda functions
        print("\n📦 Step 1: Deploying Lambda functions...")
        print("-" * 70)
        deploy_lambda_script = scripts_dir / "lambda-target-setup" / "deploy_lambdas.py"
        result = subprocess.run(
            [sys.executable, str(deploy_lambda_script)], capture_output=True, text=True
        )
        print(result.stdout)

        # Step 2: Setup Gateway
        print("\n🌐 Step 2: Setting up AgentCore Gateway...")
        print("-" * 70)
        setup_gateway_script = scripts_dir / "setup_gateway.py"
        result = subprocess.run(
            [sys.executable, str(setup_gateway_script)], capture_output=True, text=True
        )
        print(result.stdout)

        print("\n" + "=" * 70)
        print("✅ Infrastructure setup complete!")
        print("=" * 70)

        # Verify config was created
        if config_file.exists():
            print(f"✅ Configuration file created: {config_file}")
        else:
            print("⚠️  Warning: Setup completed but config.json was not created")

    except Exception as e:
        print(f"\n❌ Setup failed: {e}")
        print("\nPlease run the setup scripts manually:")
        print(f"1. cd {getting_started_dir}")
        print("2. python scripts/lambda-target-setup/deploy_lambdas.py")
        print("3. python scripts/setup_gateway.py")
    finally:
        # Change back to original directory
        os.chdir(current_dir)

print("\n" + "=" * 70)

---

# ステップ 1: Policy Engine の作成

Cedar ポリシーを保持する Policy Engine を作成します

Policy Engine はポリシーのコレクションです。Gateway に関連付けることで、インバウンドトラフィックに対するポリシーのリアルタイム適用が可能になります。
以降のステップでこの Policy Engine にポリシーを作成していきます

### Policy Engine の作成

まず、Cedar ポリシーを保持する Policy Engine を作成します：

In [ ]:
from bedrock_agentcore_starter_toolkit.operations.policy.client import PolicyClient

policy_client = PolicyClient(region_name=region)
policy_client.logger.setLevel(logging.INFO)

# Create a Policy Engine
print("🔧 Creating Policy Engine...")
engine = policy_client.create_or_get_policy_engine(
    name="InsurancePolicyEngine_NL2Cedar",
    description="Policy engine for insurance underwriting governance",
)
print(f"✓ Policy Engine: {engine['policyEngineId']}\n")

# Save Policy Engine in the configuration file
with open(config_file, "r") as f:
    config = json.load(f)

# Add policy engine information (without removing existing data)
config["policy_engine_id"] = engine["policyEngineId"]
config["policy_engine_arn"] = engine["policyEngineArn"]

# Write back the updated config
with open(config_file, "w") as f:
    json.dump(config, f, indent=2)

## ステップ 2: 自然言語からポリシーを生成

NL2Cedar 機能を使用して自然言語から Cedar ポリシーを生成します。NL2Cedar を使用したポリシー作成には 2 つのステップがあります：まず自然言語から Cedar ポリシーを生成し、次に Policy Engine にポリシーを作成します。

> **ヒント**: Gateway 上のターゲットのスキーマが NL2Cedar 機能に提供され、基盤モデルがターゲット名とパラメータを理解できるようになります。

Gateway には 3 つの Lambda ターゲットがあります：
1. Application Tool: 簡易版申請作成（デモ用のモック）
 申請者の地域と補償額を指定して保険申請を作成
 パラメータ:
 - applicant_region: 顧客の地理的地域
 - coverage_amount: 希望する保険補償額

2. Risk Model Tool: 簡易版リスクモデルアクセス（デモ用のモック）
 リスクスコアリングモデルを呼び出し、評価を返す
 パラメータ:
 - API_classification: API 分類（public、internal、restricted）
 - data_governance_approval: データガバナンスがモデル使用を承認したかどうか

3. Approval Tool: 保険承認プロセス（デモ用のモック）
 引受決定と請求額を承認
 パラメータ:
 - claim_amount: 保険請求/補償額
 - risk_level: リスクレベル評価（low、medium、high、critical）


自然言語ステートメントでは、これらのターゲットのいずれかを参照し、これらのパラメータに基づいてエージェント-ツールアクセスに制約を導入できます。

In [ ]:
# Create Cedar policy
print("\n📝 Generating Cedar Policy from Natural language...")
print("\n📝 Simple natural language statement")

nl_input = "Allow all users to invoke the application tool when the coverage amount is under 1 million and the application region is US or CAN"

result = policy_client.generate_policy(
    policy_engine_id=config["policy_engine_id"],
    name=f"nl_policy_{int(time.time())}",
    resource={"arn": config["gateway"]["gateway_arn"]},
    content={"rawText": nl_input},
    fetch_assets=True,
)

In [ ]:
if result.get("status") == "GENERATED" and result.get("generatedPolicies"):
    generated_policy = result["generatedPolicies"][0]
    cedar_statement = (
        generated_policy.get("definition", {}).get("cedar", {}).get("statement", "N/A")
    )

    print("Generated Cedar Policy:")
    print("=" * 60)
    print(cedar_statement)
    print("=" * 60)

## ステップ 2: 生成された Cedar ポリシーからポリシーを作成

In [ ]:
if result.get("status") == "GENERATED" and result.get("generatedPolicies"):
    generated_policy = result["generatedPolicies"][0]
    application_creation_policy = policy_client.create_policy(
        policy_engine_id=config["policy_engine_id"],
        name="application_creation_policy",
        description="Allow application creation when coverage is under $1M and region is US or CA",
        definition=generated_policy.get("definition", {}),
    )

---

# 自然言語からのその他のポリシー生成タイプ

### 1. 複数行ステートメント
複数行ステートメントが提供された場合、複数のポリシーが生成され、result['generatedPolicies'] に含まれます。一貫して現れる区切り文字（カンマ、ピリオド、セミコロンなど）が個々のポリシーステートメントを区別するために使用されます

In [ ]:
print("\n📝 Multi-line statement")

nl_input = """Allow all users to invoke the risk model tool when data governance approval is true. 
Block users from calling the application tool unless coverage amount is present"""

result = policy_client.generate_policy(
    policy_engine_id=config["policy_engine_id"],
    name=f"nl_policy_{int(time.time())}",
    resource={"arn": config["gateway"]["gateway_arn"]},
    content={"rawText": nl_input},
    fetch_assets=True,
)

if result.get("status") == "GENERATED" and result.get("generatedPolicies"):
    for generated_policy in result["generatedPolicies"]:
        cedar_statement = (
            generated_policy.get("definition", {})
            .get("cedar", {})
            .get("statement", "N/A")
        )

        print("Generated Cedar Policy:")
        print("=" * 60)
        print(cedar_statement)
        print("=" * 60)

### 2. プリンシパル関連のポリシーステートメント
プリンシパルスコープに基づいて条件を主張するポリシーを作成できます。これは OAuth アクセストークンの形式で IdP 表現を通じて中継されます。OAuth トークンに格納される属性はカスタム設定可能であるため、NL2Cedar 生成では正確なタグを提供することで、正しい Cedar ポリシーを作成できます

In [ ]:
print("\n📝 Principal Scope statements")

nl_inputs = [
    'Allow principals with username "test-user" to invoke the risk model tool',
    str(
        'Forbid principals to access the approval tool unless they have the scope group:Controller <idp_claims>["scope"]</idp_claims>'
    ),
    str(
        'Block principals from using risk model tool and approval tool unless the principal has role "senior-adjuster"'
    ),
]

for nl_input in nl_inputs:
    result = policy_client.generate_policy(
        policy_engine_id=config["policy_engine_id"],
        name=f"nl_policy_{int(time.time())}",
        resource={"arn": config["gateway"]["gateway_arn"]},
        content={"rawText": nl_input},
        fetch_assets=True,
    )

    if result.get("status") == "GENERATED" and result.get("generatedPolicies"):
        for generated_policy in result["generatedPolicies"]:
            cedar_statement = (
                generated_policy.get("definition", {})
                .get("cedar", {})
                .get("statement", "N/A")
            )
            print("=" * 60)
            print(f"自然言語: {nl_input}")
            print("Generated Cedar Policy:")
            print("=" * 60)
            print(cedar_statement)
            print("=" * 60)

# クリーンアップ

In [ ]:
from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient
from bedrock_agentcore_starter_toolkit.operations.policy.client import PolicyClient

with open(config_file, "r") as f:
    config = json.load(f)

# Clean up Policy Engine first
print("🧹 Cleaning up Policy Engine...")
policy_client = PolicyClient(region_name=config["region"])
policy_client.cleanup_policy_engine(config["policy_engine_id"])
print("✓ Policy Engine cleaned up\n")

# Then clean up Gateway
print("🧹 Cleaning up Gateway...")
gateway_client = GatewayClient(region_name=config["region"])
gateway_client.cleanup_gateway(
    config["gateway"]["gateway_id"], config["gateway"]["client_info"]
)
print("✅ Cleanup complete!")